In [27]:
# Criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd

#Criar o navegador 
nav = webdriver.Chrome()

# Importar/visualizar a base de dados
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)



,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [28]:
#1. Entrar no google
nav.get("https://www.google.com.br/")

produto = 'iphone 12 64 gb'
termos_banidos ='mini watch'

produto = produto.lower()
termos_banidos = termos_banidos.lower()
lista_termos_banidos = termos_banidos.split(" ")
lista_termos_produto = produto.split(" ")

preco_minimo = 4000
preco_maximo = 4500

#2. Pesquisar o nome do produto no google
nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#3. Clicar na aba shopping
elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
for item in elementos:
    if 'Shopping' in item.text:
        item.click()
        break

#4. Pegar todos os elementos dos produtos no shopping
lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')


# para cada item dentro da nossa base de dados (para cada produto)
    #procurar esse produto no google shopping
        #verificar se algum dos produtos do google shopping está dentro da minha faixa de preço
    #procurar esse produto no buscapé
        #verificar se algum dos produtos do buscape está dentro da minha faixa de preço

# salvar as ofertas boas em um dataframe (tabela)
# exportar pro excel
# enviar por e-mail o resultado da tabela.

In [30]:
lista_ofertas = []
for resultado in lista_resultados:

    nome = resultado.find_element(By.CLASS_NAME,'Xjkr3b').text
    nome = nome.lower()

    #Verificação do Nome
    
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    
    tem_todos_termos_produto = True
    for palavra in lista_termos_produto:
        if palavra not in nome:
            tem_todos_termos_produto = False
    
    
    #Se tem_termos_banidos = False e o tem_todos_termos_produto = True, então executa análise 
    if not tem_termos_banidos and tem_todos_termos_produto: # Verificando o nome
        preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
        preco = preco.replace('R$', '').replace(' ','').replace('.','').replace(',','.')
        preco = float(preco)

        #verificando e o preco está dentro do minimo e maximo
        preco_maximo = float(preco_maximo)
        preco_minimo = float(preco_minimo)
        if preco_minimo <= preco <= preco_maximo:
             #pegando o elemento que está abaixo do que queremos obter
            elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
            #subindo para pegar o elemento que queremos
            elementos_pai = elemento_link.find_element(By.XPATH, '..')
            #Pegando de fato o link de cada produto
            link = elementos_pai.get_attribute('href')
            lista_ofertas.append((nome, preco, link))
            

In [31]:
print(lista_ofertas)

[('apple iphone 12, 64gb - branco', 4499.0, 'https://www.google.com.br/url?url=https://www.fastshop.com.br/web/p/d/AEMGJ63BRABCO_PRD/iphone-12-branco-64gb-mgj63bra-fast/%3Fpartner%3Dparceiro-google%26cm_mmc%3Dcpc_Shopping-_-AEMGJ63BRABCO_PRD&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiuyIHYhPb2AhWeHrkGHeASB64Q_uQECN4L&usg=AOvVaw3T4vpWiTwU06SqxBllHdab'), ('iphone 12 64gb roxo, com tela de 6,1, 5g e câmera dupla de 12 mp - mjnm3br/a', 4499.0, 'https://www.google.com.br/url?url=https://www.fastshop.com.br/web/p/d/AEMJNM3BRARXO_PRD/iphone-12-roxo-64gb-mjnm3br-fast/%3Fpartner%3Dparceiro-google%26cm_mmc%3Dcpc_Shopping-_-AEMJNM3BRARXO_PRD&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiuyIHYhPb2AhWeHrkGHeASB64QguUECLAM&usg=AOvVaw3UV9Zr2hAoy8ZQ6KP8oAR2'), ('iphone 12 - 64 gb, branco', 4390.03, 'https://www.google.com.br/url?url=https://wantimport.com/loja/apple/iphone/iphone-12/%3Fattribute_pa_gb%3D64-gb%26attribute_pa_cor%3Dbranco&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiuyIHYhPb2AhWeHrkGHeASB64QgOUECJ4P&usg=AOvVaw1BTySNsrTgJ